In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = pd.read_excel("proj1Dataset.xlsx")
data.__class__

pandas.core.frame.DataFrame

In [5]:
data.dropna()
data.head(5)

,Weight,Horsepower
0,3504,130.0
1,3693,165.0
2,3436,150.0
3,3433,150.0
4,3449,140.0


### Weight = x-axis
### Horsepower = y-axis

for closed form:
1. 
